In [28]:
from dotenv import load_dotenv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from bs4 import BeautifulSoup
import pandas as pd
load_dotenv()

True

In [32]:
# grab the zip code we've saved as an environment variable
zip_code = os.environ.get('ZIP')

driver = webdriver.Firefox()
driver.maximize_window()
# access Whole Foods' products that are on sale via selenium
driver.get("https://www.wholefoodsmarket.com/products/all-products?featured=on-sale")
assert "Whole Foods" in driver.title

# wait for the page to load
wait = WebDriverWait(driver, 3)

# try/except in case selenium opens us logged in for some reason
try:
    # we create a variable to grab the search field
    store = driver.find_element(by=By.ID, value="pie-store-finder-modal-search-field")
    # ensure it's empty
    store.clear()
    # selenium adds our zip code into the text field
    store.send_keys(zip_code)
    store.send_keys(Keys.RETURN)
    
    # wait until a nearby Whole Foods shows up as a clickable option
    closest = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'li.wfm-search-bar--list_item:nth-child(1)')))
    closest.click()
except:
    NoSuchElementException('Either you already have the lcoation or there\'s an error')

# the on sale products page does not show all of the products
# in fact, there are so many deals on sale that I have no idea
# what the exact amount is, so I limit the amount of extra
# product pages that we load more to 5

# we wait for the load more button to load since it doesn't initially
load_more = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button.w-button:nth-child(5)")))
for _ in range(5):
    # we click 5 times
    load_more.click()

In [43]:
# grabbing the html source after loading more deals
# than we would have gotten after initially loading
# the page
content = driver.page_source

# make working with the html easier with BeautifulSoup
dom = BeautifulSoup(content, 'html.parser')
body = dom.body

In [62]:
# select the products by grabbing all content in the body
# that have the class for a product item

# we need to use a dictionary to select this css atrribute
# because python already has a class keyword
products = body.find_all(attrs={'class': 'w-pie--product-tile'})
print(products[0].prettify())

<div class="w-pie--product-tile" data-testid="product-tile">
 <a class="w-pie--product-tile__link" data-csa-c-content-id="ASIN: B0787WYH81" data-csa-c-id="lnumb5-d70svo-wqcama-3r3kqw" data-csa-c-slot-id="1" data-csa-c-type="productTile" data-testid="product-tile-link" href="/product/whole-foods-market-beef-ribeye-steak-b0787wyh81">
  <div class="w-pie--product-tile__image">
   <div class="w-pie--product-tile__badge" data-testid="product-tile-badge">
    <span class="w-cms--font-disclaimer" data-testid="product-tile-badge-text">
    </span>
    <span class="w-pie--badge w-pie--badge__sale" data-testid="product-tile-badge-icon">
     <span class="w-visually-hidden">
      Sale Item
     </span>
    </span>
   </div>
   <img alt="Beef Ribeye Steak" class="ls-is-cached lazyloaded" data-src="https://m.media-amazon.com/images/S/assets.wholefoodsmarket.com/PIE/product/5f49863d144b13f9b96ca4dc_2020-cen-ecommerce-meat-beefribboneless._TTD_._SR300,300_.jpg" data-testid="product-tile-image" loadi

In [67]:
df = pd.DataFrame()

# loop through every product
for i in products:
    # search for elements with the pricing information
    info = i.find(attrs={'class': 'w-pie--product-tile__content'})
    
    brand = info.find(attrs={'class': 'w-cms--font-disclaimer'}).text
    item = info.find(attrs={'class': 'w-cms--font-body__sans-bold'}).text
    regular = info.find(attrs={'class': 'regular_price has_sale'}).text
    sale = info.find(attrs={'class': 'sale_price'}).text
    prime = info.find(attrs={'class': 'prime_price prime_incremental'}).text
    print(prime)
    
    """ df.append({'Brand': brand,
                'Item': item,
               'Regular': regular,
               'Sale': sale,
               'Prime': prime}) """

Whole Foods Market
PRODUCE
PRODUCE
Vital Farms
SEAFOOD
SEAFOOD
PRODUCE
Organic Valley
GREAT RANGE BISON
Maple Hill Creamery
Pete And Gerry's
SEAFOOD
Creminelli
MEAT
Whole Foods Market
PRODUCE
Kerrygold
Vital Farms
PRODUCE
Wellshire Farms
Organic Valley
Kerrygold
Maple Hill Creamery
Principe di San Daniele
Stonyfield
Kerrygold
Kerrygold
Waterloo
MEAT
GimMe
Waterloo
Organic Valley
Tasteful Selections
Fulvi
MALK Organics
La Panzanella
Organic Valley
LesserEvil
Kerrygold
Waterloo
SmartSweets
GT's Synergy Kombucha
Waterloo
Whole Foods Market
GT's Synergy Kombucha
MEAT
Chameleon Cold-Brew
Wildwood
Whole Foods Market
Bubbies
Stonyfield
Alden's Organic Ice Cream
Kevin's Natural Foods
Field Roast
Evolution Fresh
Kerrygold
Smart Water
Seggiano
Solely
GT's Cocoyo
St Andre
Organic Valley
Whole Foods Market
Organic Valley
Waterloo
GT's Cocoyo
Waterloo
siggi's
Hippeas
Kevin's Natural Foods
Catalina Crunch
Vermont Creamery
Kerrygold
Organic Valley
Lactalis
Vital Farms
GoodPop
Wildwood
Vital Farms
Cat